<a href="https://colab.research.google.com/github/aoi1224/test/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np


# Load data
pwt90 = pd.read_stata('https://www.rug.nl/ggdc/docs/pwt90.dta')
#pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt90.loc[pwt90['country'].isin(["Australia", "Austria"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh']]
data = data.loc[(data['year'] >= 1960) & (data['year'] <= 2000)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  # Capital share

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates, capital deepening and TFP growth

data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['cd'] = (grouped_data['k_pc'].diff() * 100) * (data['a'])  # capital deepening
data['t'] = data['g'] - data['cd']

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean', 'a': "mean",'labsh': "mean",'cd': "mean",'t': "mean"})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Share'] = summary['a']
summary['TFP Share'] = summary['labsh']
summary['TFP Growth'] = summary['t']
summary['Capital Deepening'] = summary['cd']

# Print output
print(summary)

                    g         a     labsh        cd         t  Growth Rate  \
countrycode                                                                  
AUS          1.724265  0.348140  0.651860  0.564797  1.159467     1.724265   
AUT          3.038354  0.365929  0.634071  1.409189  1.629165     3.038354   

             Capital Share  TFP Share  TFP Growth  Capital Deepening  
countrycode                                                           
AUS               0.348140   0.651860    1.159467           0.564797  
AUT               0.365929   0.634071    1.629165           1.409189  
